In [24]:
import pickle
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 200

data = None
regex_decimal = r'(\d+)'
regex_float = r'([-+]?\d*\.*\d+)'
regex_float_with_dec_part = r'(\d+\.\d+)'

# Raw data, source: mediamarkt.es, web scraping 
# Working on the backup version from 19.03.2020 to provide that the code interprets without errors. 
# The set is updated frequently and adjustments may be needed when used with new phones.
with open('data_raw_backup.pickle', 'rb') as handle:
    smartphones = pickle.load(handle) # list of dictionaries, each dictionary describes one smartphone
    data = pd.DataFrame.from_dict(smartphones)

In [25]:
# Matching columns to slot names in Clips 'smartphone' template
columns = {'Marca' : "brand",
          'Tipo:' : "model",
          'Sistema operativo:' : "operating_system",
          'Velocidad Procesador:' : "cpu",
          'Capacidad memoria:' : "memory",
          'Tamaño memoria RAM:' : "ram",
          'Resolución cámara trasera:' : "camera",
          'Peso:' : "weight",
          'Tamaño pantalla (cm):' : "screen_diagonal",
          'Capacidad batería:' : "battery",
          'Precio' : "price",
          'Admite SD:' : "sd",
          'GPS:' : "gps",
          'Bluetooth:' : "bluetooth",
          'Wi-Fi:' : "wifi",
          'Link' : "link"
          }

In [26]:
# Filtering the columns to eliminate unused ones and renaming
data = data[list(columns)]
data = data.rename(columns=columns)
data.head()

,brand,model,operating_system,cpu,memory,ram,camera,weight,screen_diagonal,battery,price,sd,gps,bluetooth,wifi,link
0,huawei,P30 Lite,Android,2.2 GHz,128 GB,4 GB,48 megapixel,159 g,15.62 cm,3340 mAh,"229,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-huawei-p30-lite-azul-128-gb-4-gb-ram-6-15-full-hd-kirin-710-3340-mah-android-1452688.html
1,xiaomi,Redmi Note 8 Pro,Android,"2,05 GHz",128 GB,6 GB,64 pixels,200 g,NaN,4500 mAh,"228,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8-pro-azul-128-gb-6-gb-ram-6-53-full-hd-helio-g90t-4500-mah-android-1469588.html
2,xiaomi,Redmi Note 8T,Android,2.0 GHz,64 GB,4 GB,NaN,190 g,16 cm,4000 mAh,"199,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-azul-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468593.html
3,xiaomi,Mi 9,Android,2.84 GHz,128 GB,6 GB,48 megapixel,173 g,16.25 cm,3300 mAh,"369,-",no,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-mi-9-azul-128-gb-6-gb-ram-6-4-full-hd-snapdragon-855-3300-mah-android-1447026.html
4,xiaomi,Redmi Note 8T,Android,2.0 GHz,64 GB,4 GB,48 megapixel,190 g,16 cm,4000 mAh,"199,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-gris-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468591.html


In [27]:
# Brand
# Hand-check if values overlap due to unconsistent spelling
data.brand.value_counts()

apple        75
samsung      54
huawei       31
lg           21
sony         19
honor        19
xiaomi       17
motorola     14
hammer       10
alcatel       9
oppo          9
nokia         6
crosscall     3
realme        2
tcl           2
meizu         2
palm          1
gigaset       1
asus          1
weimei        1
moto          1
vsmart        1
lenovo        1
oneplus       1
Name: brand, dtype: int64

In [28]:
# Operating system
# replace "Android 6.0" by "Android"
data.loc[data.operating_system == "Android 6.0", "operating_system"] = "Android"
# replace "Propietario" by "other"
data.loc[data.operating_system == "Propietario", "operating_system"] = "other"
data.operating_system.value_counts()

Android    200
iOS         74
other        1
Name: operating_system, dtype: int64

In [29]:
# CPU
# Identified units: GHz|MHz|nm. 
# One value doesn't have unit -> hand edit
data.loc[data.cpu == "1.3", "cpu"] = "1.3 GHz"
# assert that all values have identified units
assert data.cpu.str.contains(pat='GHz|MHz|nm', na=True).all()
data.cpu = data.cpu.str.replace(',', '.')
# mark MHz and nm
data['cpu_MHz'] = data.cpu.str.contains(pat='MHz', na=False)
data['cpu_nm'] = data.cpu.str.contains(pat='nm', na=False)
# mark multiple cores cpu
data['cpu_mult'] = data.cpu.str.contains(pat='x|-', na=False)
# extract largest value from multiple cores cpu
data.cpu = data.apply(lambda x: max(re.findall(regex_float_with_dec_part,x.cpu)) if x.cpu_mult is True else x.cpu, axis=1)
# change values to float
data.cpu = data.cpu.str.extract(regex_float)
data.cpu = data.apply(lambda x: pd.to_numeric(x.cpu), axis=1)
# change MHz to GHz
data.cpu = data.apply(lambda x: x.cpu/1000 if x.cpu_MHz is True else x.cpu, axis=1)
# change nm to negative number, eg. -7
data.cpu = data.apply(lambda x: -x.cpu if x.cpu_nm is True else x.cpu, axis=1)
data.pop('cpu_MHz')
data.pop('cpu_nm')
data.pop('cpu_mult')
data.shape

(301, 16)

In [30]:
# Memory
# Most values GB, some MB exist
# assert that all values are either MB or GB
assert data.memory.str.contains(pat='MB|GB', na=True).all()

data['memory_mb'] = data.memory.str.contains(pat='MB', na=False)
data.memory = data.memory.str.extract(regex_decimal)
data.memory = pd.to_numeric(data.memory)
# for each row containing MB calculate to GB
data.memory = data.apply(lambda x: x.memory / 1024 if x.memory_mb is True else x.memory, axis=1)
data.pop('memory_mb')
data.shape

(301, 16)

In [31]:
# RAM
# similar as memory treatment
assert data.ram.str.contains(pat='MB|GB', na=True).all()

data['ram_mb'] = data.ram.str.contains(pat='MB', na=False)
data.ram = data.ram.str.extract(regex_decimal)
data.ram = pd.to_numeric(data.ram)
data.ram = data.apply(lambda x: x.ram / 1024 if x.ram_mb is True else x.ram, axis=1)
data.pop('ram_mb')
data.shape

(301, 16)

In [32]:
# Camera
# Identified units: megapixel, pixels (refers to megapixels)
# Assert that all entries have identified units
assert data.camera.str.contains(pat='megapixel|pixels', na=True).all()
data.camera = data.camera.str.extract(regex_decimal)
data.camera = data.camera.astype('float')
data.shape

(301, 16)

In [33]:
data.camera

0       48.0
1       64.0
2        NaN
3       48.0
4       48.0
5       16.0
6        NaN
7        NaN
8       13.0
9       16.0
10      12.0
11      64.0
12      40.0
13      12.0
14      40.0
15      12.0
16      12.0
17      12.0
18       NaN
19      48.0
20      12.0
21      12.0
22       NaN
23       NaN
24      12.0
25       NaN
26      48.0
27      13.0
28      13.0
29      13.0
30      48.0
31      12.0
32      13.0
33      12.0
34      12.0
35      13.0
36      64.0
37       NaN
38       NaN
39       NaN
40      13.0
41      12.0
42       5.0
43      13.0
44      64.0
45      48.0
46       NaN
47      12.0
48      25.0
49      16.0
50     108.0
51     108.0
52     108.0
53      13.0
54      12.0
55       NaN
56      12.0
57      13.0
58       NaN
59      12.0
60      12.0
61      12.0
62      64.0
63      12.0
64      48.0
65      12.0
66       NaN
67       NaN
68      12.0
69      12.0
70      12.0
71      12.0
72      12.0
73       NaN
74      12.0
75      48.0
76      13.0

In [34]:
# Weight
# Identified units: g, gr, kg (=1000g)
# Assert that all entries have identified units
assert data.weight.str.contains(pat='g|gr|kg', na=True).all()
data['weight_kg'] = data.weight.str.contains(pat='kg', na=False)
data.weight = data.weight.str.extract(regex_float)
data.weight = pd.to_numeric(data.weight)
# for each row containing kg calculate to g
data.weight = data.apply(lambda x: x.weight * 1000 if x.weight_kg is True else x.weight, axis=1)
data.pop('weight_kg')
# assert that all values (drop NaN) are within reasonable range
assert data.weight.dropna().between(100,300).all()
data.shape

(301, 16)

In [35]:
# Screen diagonal
# Identified units: cm
# Assert that all entries have identified units
assert data.screen_diagonal.str.contains(pat='cm', na=True).all()
data.screen_diagonal = data.screen_diagonal.str.extract(regex_float)
data.screen_diagonal = pd.to_numeric(data.screen_diagonal)
# assert that all values (drop NaN) are within reasonable range
assert data.screen_diagonal.dropna().between(5,21).all()
data.shape

(301, 16)

In [36]:
# Battery
# Identified units: mAh
assert data.battery.str.contains(pat='mAh', na=True).all()
data.battery = data.battery.str.extract(regex_decimal)
data.battery = pd.to_numeric(data.battery)
# assert that all values (drop NaN) are within reasonable range
assert data.battery.dropna().between(0,8000).all()
data.shape

(301, 16)

In [37]:
# Price
data.price = data.price.str.replace(',', '.')
data.price = data.price.str.extract(regex_decimal)
data.price = pd.to_numeric(data.price)
# assert that all values (drop NaN) are within reasonable range AND that all values are given
assert data.price.dropna().between(0,4000).all()
assert not(data.price.isna().all())
data.shape

(301, 16)

In [38]:
# Binary columns: sd gps bluetooth wifi
binary_columns = [data.sd, data.gps, data.bluetooth, data.wifi]
pd.concat(binary_columns).value_counts()
# Identified values: Sí, no
def fun(x): 
 return x.str.replace('Sí', 'y').replace('no', 'n')
data.sd = fun(data.sd)
data.gps = fun(data.gps)
data.bluetooth = fun(data.bluetooth)
data.wifi = fun(data.wifi)
data.shape

(301, 16)

In [39]:
data.head()

,brand,model,operating_system,cpu,memory,ram,camera,weight,screen_diagonal,battery,price,sd,gps,bluetooth,wifi,link
0,huawei,P30 Lite,Android,2.20,128.0,4.0,48.0,159.0,15.62,3340.0,229,y,y,y,y,/es/product/_m%C3%B3vil-huawei-p30-lite-azul-128-gb-4-gb-ram-6-15-full-hd-kirin-710-3340-mah-android-1452688.html
1,xiaomi,Redmi Note 8 Pro,Android,2.05,128.0,6.0,64.0,200.0,NaN,4500.0,228,y,y,y,y,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8-pro-azul-128-gb-6-gb-ram-6-53-full-hd-helio-g90t-4500-mah-android-1469588.html
2,xiaomi,Redmi Note 8T,Android,2.00,64.0,4.0,NaN,190.0,16.00,4000.0,199,y,y,y,y,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-azul-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468593.html
3,xiaomi,Mi 9,Android,2.84,128.0,6.0,48.0,173.0,16.25,3300.0,369,n,y,y,y,/es/product/_m%C3%B3vil-xiaomi-mi-9-azul-128-gb-6-gb-ram-6-4-full-hd-snapdragon-855-3300-mah-android-1447026.html
4,xiaomi,Redmi Note 8T,Android,2.00,64.0,4.0,48.0,190.0,16.00,4000.0,199,y,y,y,y,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-gris-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468591.html


In [41]:
data.price[0].dtype

dtype('int64')

In [42]:
with open('data_clean.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.DEFAULT_PROTOCOL)

Save values to use in the interface (border values for ranges and selectable values (sperating systems, brands)

In [55]:
ranges = {}
ranges["min"] = {}
ranges["max"] = {}

from math import ceil

ranges["min"]["cpu"] = float(data.cpu[data.cpu > 0].min())
ranges["max"]["cpu"] = float(data.cpu[data.cpu > 0].max())

ranges["min"]["memory"] = int(data.memory.min())
ranges["max"]["memory"] = int(data.memory.max())

ranges["min"]["ram"] = int(data.ram.min())
ranges["max"]["ram"] = int(data.ram.max())

ranges["min"]["camera"] = float(data.camera.min())
ranges["max"]["camera"] = float(data.camera.max())

ranges["min"]["weight"] = int(data.weight.min())
ranges["max"]["weight"] = int(data.weight.max())

ranges["min"]["diagonal"] = float(data.screen_diagonal.min())
ranges["max"]["diagonal"] = float(data.screen_diagonal.max())

ranges["min"]["battery"] = int(data.battery.min())
ranges["max"]["battery"] = int(data.battery.max())

ranges["min"]["price"] = int(data.price.min())
ranges["max"]["price"] = int(data.price.max())

In [56]:
ranges["brands"] = data.brand.unique()
ranges["os"] = data.operating_system.unique()

In [57]:
ranges

{'min': {'cpu': 0.416,
  'memory': 0,
  'ram': 0,
  'camera': 2.0,
  'weight': 113,
  'diagonal': 6.09,
  'battery': 0,
  'price': 49},
 'max': {'cpu': 3.0,
  'memory': 512,
  'ram': 16,
  'camera': 108.0,
  'weight': 300,
  'diagonal': 18.51,
  'battery': 5260,
  'price': 2599},
 'brands': array(['huawei', 'xiaomi', 'samsung', 'apple', 'realme', 'motorola',
        'honor', 'lg', 'alcatel', 'oneplus', 'oppo', 'gigaset', 'sony',
        'crosscall', 'hammer', 'nokia', 'palm', 'asus', 'meizu', 'tcl',
        'weimei', 'moto', 'vsmart', 'lenovo'], dtype=object),
 'os': array(['Android', 'iOS', 'other', nan], dtype=object)}

In [58]:
with open('border_values.pickle', 'wb') as handle:
    pickle.dump(ranges, handle, protocol=pickle.DEFAULT_PROTOCOL)

In [22]:
ranges["max"]["cpu"]

3.0

In [23]:
data.cpu.max()

3.0